In [29]:
# Load the path from Valhalla

import requests
import json

def get_distance(pointA, pointB):
    url = "http://valhalla:8002/route"
    data =  {
        "locations":[
            {
                "lat":pointA[1],
                "lon":pointA[0],
                "type":"break"
            },{
                "lat":pointB[1],
                "lon":pointB[0],
                "type":"break"
            }
        ],
        "costing":"pedestrian",
        "directions_options":{
            "units":"miles"
        }
    }

    data_json = json.dumps(data)
    response = requests.post(url, data=data_json)
    response_obj = json.loads(response.text)
    #distance = response_obj['trip']['summary']['length']
    time_in_seconds = response_obj['trip']['summary']['time']
    #west = response_obj['trip']['summary']['min_lon']
    #south = response_obj['trip']['summary']['min_lat']
    #east = response_obj['trip']['summary']['max_lon']
    #north = response_obj['trip']['summary']['max_lat']
    #geometry = decode_geometry(response_obj['trip']['legs'][0]['shape'])
    return time_in_seconds/60

In [32]:
import psycopg2
conn = psycopg2.connect(host="postgres",database="osm", user="osm", password="osm")
cursor = conn.cursor()
lat = 40.222884
lon = -74.010232
distance_in_meters = 1000
amenity = 'restaurant'
cursor.execute("""
select * FROM (
    select
     name,
     ST_X(ST_Transform(way, 4326)) as lon,
     ST_Y(ST_Transform(way, 4326)) as lat,
     ST_DISTANCE(
         way,
         ST_TRANSFORM(
             ST_SETSRID(
                 ST_GeomFromText(
                     'POINT(%f %f)'
                 ),
             4326),
         3857)
     ) AS distance
    FROM
       planet_osm_point
    WHERE
      amenity = '%s'
) AS subquery WHERE distance < %f 
ORDER BY distance;
""" % (lon, lat, amenity, distance_in_meters))
pointA = [lon, lat]
for record in cursor:
    pointB = [record[1], record[2]]
    time = get_distance(pointA, pointB)
    print (record[0], "(%f min)" % time, record[3])
    
cursor.close()


Jonti S Latin & Soul Food (6.900000 min) 760.528083688581
Pascal & Sabine (9.016667 min) 852.559714144463
Cardinal Provisions (9.900000 min) 880.965369533937
Barrio Costero (9.116667 min) 893.780302984483
The Bonney Read (10.666667 min) 929.381059187401
Lola's European Cafe (10.900000 min) 944.201640732824
Cross & Orange (11.066667 min) 956.12091627585
Asbury Ale House (10.566667 min) 957.914888938367
Toast Asbury Park (10.900000 min) 959.043693106551
Brickwall Tavern & Dining Room (10.700000 min) 963.891752771227
DJ's Delights Deli & Restaurant (9.566667 min) 988.894427663788
Twisted Tree Cafe (10.666667 min) 999.968517183353
